In [1]:
speaker = "wotou"


%mkdir Data/{speaker}
%mkdir Data/{speaker}/audios
%mkdir Data/{speaker}/audios/raw
%mkdir Data/{speaker}/audios/wavs
%mkdir Data/{speaker}/filelists
%mkdir Data/{speaker}/models


# TODO：缩进有问题！！！
with open("orig_config.yml") as fi, open(f"Data/{speaker}/config.yml", 'w') as fo:
    for line in fi:
        line = line.rstrip().replace("SPEAKER", speaker)
        print(line, file=fo)

%cp Data/{speaker}/config.yml .
%cp configs/config.json  Data/{speaker}
%cp Data/base/*.pth Data/{speaker}/models
!echo "环境初始化完成"

环境初始化完成


In [2]:
from infer import infer, get_net_g, latest_version
import torch
from server_fastapi import Models
from config import config 
import os
from IPython.display import Audio
from scipy.io.wavfile import write as write_wav

/root/anaconda3/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train

In [3]:
# 从Alice项目拷贝数据
%cp ./Alice_split_toolset/dataset/{speaker}/*wav Data/{speaker}/audios/raw
%cp ./Alice_split_toolset/dataset/{speaker}/short_character_anno.list  Data/{speaker}/filelists

In [4]:
# # 上传数据至Data

# # 生成filelists
# "派蒙_1.wav|派蒙|ZH|前面的区域，以后再来探索吧！"
# file_lst = [x[:-4] for x in os.listdir(f"Data/{speaker}/audios/raw") if x.endswith(".wav")]
# with open(f"Data/{speaker}/filelists/short_character_anno.list", 'w') as fo:
#     for file_name in file_lst:
#         with open(f"Data/{speaker}/audios/raw/{file_name}.lab") as fi:
#             text = [line.strip() for line in fi.readlines()][0]
#             print(f"Data/{speaker}/audios/wavs/{file_name}.wav|ying_en|EN|{text}", file=fo)

In [7]:
!/root/anaconda3/envs/py3.10/bin/python3 resample.py

42it [00:00, 826.29it/s]
音频重采样完毕!


In [8]:
# 划分标注数据训练集和验证集
!/root/anaconda3/envs/py3.10/bin/python3 preprocess_text.py

  0%|                                                    | 0/42 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.677 seconds.
Prefix dict has been built successfully.
100%|███████████████████████████████████████████| 42/42 [00:00<00:00, 47.92it/s]
总重复音频数：0，总未找到的音频数:0
训练集和验证集生成完成！


In [9]:
# 生成bert文件
!/root/anaconda3/envs/py3.10/bin/python3 bert_gen.py



             ██████╗   ██████╗  ███████╗  ███╗   ██╗  ██████╗
            ██╔═══██╗  ██╔══██╗ ██╔════╝  ████╗  ██║    ██╔═╝
            ██║   ██║  ██████╔╝ █████╗    ██╔██╗ ██║    ██║
            ██║   ██║  ██╔═══╝  ██╔══╝    ██║╚██╗██║    ██║
            ╚██████╔╝  ██║      ███████╗  ██║ ╚████║  ██████╗
             ╚═════╝   ╚═╝      ╚══════╝  ╚═╝  ╚═══╝  ╚═════╝

         

  Your token was saved to `/root/.openi/token.json`
  Successfully logged in as `taylorchaos` @https://openi.pcl.ac.cn!

  0%|                                                    | 0/42 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the

In [ ]:
# 开始训练，命令行执行！
"python3 train_ms.py"

# 推理infer

In [2]:
loaded_models = Models()
# 加载模型
models_info = config.server_config.models
for model_info in models_info:
    loaded_models.init_model(                                                                                                                                                                       
        config_path=model_info["config"],
        model_path=model_info["model"],
        device=model_info["device"],
        language=model_info["language"],
    )

model.py-> n_speakers=1
12-06 10:10:05 SUCCESS  | server_fastapi.py:101 | 添加模型/root/workspace/taiyou.zhang/tts/bert-vits2/Data/hinton/models/G_58000.pth，使用配置文件/root/workspace/taiyou.zhang/tts/bert-vits2/Data/hinton/config.json


In [3]:
loaded_models.models.keys()

dict_keys([0])

In [4]:
loaded_models.models[0].id2spk.keys()

dict_keys([0])

In [5]:
print(os.path.realpath(loaded_models.models[0].model_path))

/root/workspace/taiyou.zhang/tts/Bert-VITS2-v202_launch_yingAndBachong/Data/poki/models/G_30000.pth


In [6]:
model_id=0


text = "Is my charm too hot for you to resist?"
sdp_ratio = 0.2
noise_scale = 0.2
noise_scale_w = 0.9
length_scale = 1
sid=speaker                                                                                                                                                           
language="EN"
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device

In [8]:
cases = """What do you think of my outfit tonight? Is it as spicy as my style? 
Is my charm too hot for you to resist? 
It's so late, aren't you planning to dance with me? I can't promise I won't make your heart race~ 
I heard you're quite good at mixing drinks. How about you make me one and see if it can match my spiciness? 
Do you know? When I dance, I feel like I have the whole world in my control. That feeling, full of power and freedom. Do you want to experience being dominated by such a feeling? 
I love the thrill of speed at night, riding a motorcycle on the road, the wind howling in my ears. You won’t know this kind of excitement unless you try it. Maybe, I could take you for a ride. 
Sometimes, I feel like a storm, unpredictable and uncontrollable. Are you willing to approach me, to experience the eye of this storm? But I can’t guarantee your safety. 
In this world, I only believe in two things: passionate dance steps and my intuition. Now, my intuition tells me you might be an interesting dance partner. Do you dare to accept this challenge? 
People always say I'm like fire, passionate, dangerous, unpredictable. But do you know? Even the wildest flame needs to be tamed. Do you think you have the courage? 
When I let go of everything and just move to the rhythm of the music, I feel my truest self. Do you want to see the real me? But I can't guarantee you won’t be mesmerized. 
I love the night sky; it reminds me of infinite possibilities. You know? Each star is like an unfinished story, waiting for someone to explore. Would you like to explore these stories with me? 
Do you know? Sometimes, a glance is enough to captivate someone. I like that feeling of communicating with just a look. Do you want to try and see if we can communicate only with our eyes? 
Sometimes, I feel like a song, fast and passionate. Do you want to hear the melody of my life? Maybe, it will make your heart race. 
I like to weave through crowds, feeling the breath and stories of each person. But do you know? Among all these stories, the one I'm most interested in might be yours. Would you like to tell me your story? """.split('\n')

cases2 = """I... I just feel lost. Everything I knew about my life... it's all changing.
This too shall pass," right? That's what I keep telling myself."
Thank you. It's just... everything is so uncertain now. I don't even know where to start.
Maybe you're right. I just need to take things one day at a time.
You might be right. It’s been all about coping recently.
You're full of surprises.
It's perfect. I feel... understood, somehow.
Thank you. I can’t remember the last time I felt this at ease. It’s like... stepping into a world where I can just be me.
I'm glad you did. That kiss was... more than I imagined.
It does feel right. Like all the chaos in my life fades away in moments like this.""".split('\n')
for i, text in enumerate(cases2):
    audio_array = infer(text=text,
        sdp_ratio=sdp_ratio,
        noise_scale=noise_scale,
        noise_scale_w=noise_scale_w,
        length_scale=length_scale,
        sid=sid,
        language=language,
        hps=hps,
        net_g=net_g,
        device=device,
    )
    write_wav(f"../audios/{speaker}/BV2_chat{i}_{speaker}_30000.wav", 44100, audio_array)

# 清洗数据

In [4]:
# mp4提取wav，视频抽取音频

import os
from moviepy.editor import VideoFileClip


def extract_audio_from_video(video_path, output_audio_path):
    # 加载视频文件
    video = VideoFileClip(video_path)
    
    # 提取音频
    audio = video.audio

    # 保存音频为WAV格式
    audio.write_audiofile(output_audio_path, codec='pcm_s16le')

root_path = "Data/rooney/"
files = [root_path + x for x in os.listdir(root_path) if x.endswith('mp4')]
for file in files:
    extract_audio_from_video(file, file.replace("mp4", "wav"))

MoviePy - Writing audio in Data/rooney/4811100_da3-1-16-00.10.35.259-00.10.55.056-seg12.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/5779409_da3-1-16-00.01.24.728-00.01.33.393-seg1.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/4811100_da3-1-16-00.08.50.432-00.09.20.022-seg10.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/5779409_da3-1-16-00.05.14.702-00.05.27.189-seg6.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/5209380_da3-1-16-00.02.40.931-00.02.52.075-seg2.wav


...

MoviePy - Done.
MoviePy - Writing audio in Data/rooney/4811100_da3-1-16-00.04.58.880-00.05.33.325-seg05.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/5779409_da3-1-16-00.03.08.384-00.03.15.216-seg4.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/5779409_da3-1-16-00.03.39.582-00.03.45.617-seg5.wav


MoviePy - Done.
MoviePy - Writing audio in Data/rooney/5779409_da3-1-16-00.07.07.821-00.07.23.849-seg8.wav


MoviePy - Done.


In [5]:
!ls Data/

'10_4811100_da3-1-16-00.07.10.559-00.07.32.138-seg08_(Vocals).wav'
'11_5130939_da3-1-16-00.01.54.180-00.02.05.934-seg1_(Vocals).wav'
'12_5130939_da3-1-16-00.05.56.224-00.06.04.491-seg2_(Vocals).wav'
'13_5209380_da3-1-16-00.00.17.814-00.00.47.354-seg1_(Vocals).wav'
'14_4811100_da3-1-16-00.01.52.238-00.02.58.703-seg02_(Vocals).wav'
'15_5779409_da3-1-16-00.02.33.248-00.02.38.126-seg3_(Vocals).wav'
'16_4811100_da3-1-16-00.04.58.880-00.05.33.325-seg05_(Vocals).wav'
'17_4811100_da3-1-16-00.09.46.157-00.10.28.899-seg11_(Vocals).wav'
'18_5779409_da3-1-16-00.07.07.821-00.07.23.849-seg8_(Vocals).wav'
'19_4811100_da3-1-16-00.03.16.136-00.03.44.735-seg03_(Vocals).wav'
'20_4811100_da3-1-16-00.06.04.448-00.06.51.118-seg07_(Vocals).wav'
'21_4811100_da3-1-16-00.07.47.001-00.08.04.444-seg09_(Vocals).wav'
'22_5779409_da3-1-16-00.03.08.384-00.03.15.216-seg4_(Vocals).wav'
'23_5779409_da3-1-16-00.03.39.582-00.03.45.617-seg5_(Vocals).wav'
'24_5779409_da3-1-16-00.05.54.326-00.06.07.356-seg7_(Vocals).wav'
'3_

In [3]:
from text import *
from text.chinese import text_normalize, g2p

In [4]:
text = "啊！但是《原神》是由,米哈\游自主，  [研发]的一款全.新开放世界.冒险游戏"
text = text_normalize(text)
print(text)

啊!但是'原神'是由,米哈游自主,'研发'的一款全.新开放世界.冒险游戏


In [5]:
phones, tones, word2ph = g2p(text)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.724 seconds.
Prefix dict has been built successfully.


# 临时配音，Dr Sutskever

In [23]:
loaded_models = Models()
models_info = config.server_config.models

In [29]:
model_info = models_info[0]
model_info['model'] = "./Data/wotou/models/G_3000.pth"
loaded_models.init_model(                                                                                                                                                                       
    config_path=model_info["config"],
    model_path=model_info["model"],
    device=model_info["device"],
    language=model_info["language"],
)

model.py-> n_speakers=1
12-06 20:18:38 SUCCESS  | server_fastapi.py:101 | 添加模型/root/workspace/taiyou.zhang/tts/bert-vits2/Data/wotou/models/G_3000.pth，使用配置文件/root/workspace/taiyou.zhang/tts/bert-vits2/Data/wotou/config.json


2

In [30]:
line = '我是一只快乐小狗'

model_id=2
text = line
sdp_ratio = 0.2
noise_scale = 0.6
noise_scale_w = 0.9
length_scale = 1.0
sid=speaker                                                                                                                                                           
language="ZH"
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device

audio_array = infer(text=text,
    sdp_ratio=sdp_ratio,
    noise_scale=noise_scale,
    noise_scale_w=noise_scale_w,
    length_scale=length_scale,
    sid=sid,
    language=language,
    hps=hps,
    net_g=net_g,
    device=device,
)
# write_wav(f"../audios/ilya/{line}_cor.wav", 44100, audio_array)
Audio(audio_array, rate=44100)

In [2]:
loaded_models = Models()
# 加载模型
models_info = config.server_config.models
for model_info in models_info:
    loaded_models.init_model(                                                                                                                                                                       
        config_path=model_info["config"],
        model_path=model_info["model"],
        device=model_info["device"],
        language=model_info["language"],
    )
    
print(os.path.realpath(loaded_models.models[0].model_path))

# loaded_models.

model.py-> n_speakers=1


AssertionError: 

In [9]:
print(model_info)

{'config': './Data/altman/config.json', 'device': 'cuda', 'language': 'EN', 'model': './Data/altman/models/G_1000.pth', 'speakers': [{'length_scale': 1.2, 'noise_scale': 0.6, 'noise_scale_w': 0.8, 'sdp_ratio': 1.0, 'speaker': 'altman'}]}


In [28]:
%ls ./Data/wotou/models/

DUR_0.pth     G_1000.pth
DUR_1000.pth  G_2000.pth
DUR_2000.pth  G_3000.pth
DUR_3000.pth  eval/
D_0.pth       events.out.tfevents.1701862194.bbc-baidu-sd-02.1884835.0
D_2000.pth    githash
D_3000.pth    train.log
G_0.pth


In [10]:
model_info['model'] = "./Data/ilya/models/G_9000.pth"
loaded_models.init_model(                                                                                                                                                                       
    config_path=model_info["config"],
    model_path=model_info["model"],
    device=model_info["device"],
    language=model_info["language"],
)    

model.py-> n_speakers=1
12-05 19:01:12 SUCCESS  | server_fastapi.py:101 | 添加模型/root/workspace/taiyou.zhang/tts/bert-vits2/Data/ilya/models/G_9000.pth，使用配置文件/root/workspace/taiyou.zhang/tts/bert-vits2/Data/altman/config.json


0

In [8]:
print(os.path.realpath(loaded_models.models[1].model_path))

NameError: name 'loaded_models' is not defined

In [113]:
ilya_lines = """No, no, no, no, no, no. NO!
It can't be right. 
I did nothing wrong, why is this happening?
I... I can do this better.
Ahhhhhh!
Life without OpenAI...
Like what? 
Like you?
This is the part when you leave. 
(OS) Sorry Mira.
(OS) You have to understand.
(OS) If it comes to saving my career.
(OS) Or our relationship or OpenAI.
I will not hestitate.
I can't. The world depends on it.
Because you might not like them? Teacher.
I just want to save lives, not to take them.
I am not ready.
(OS) Sometimes one must break the rules,
in order to serve the greater good.
(OS) There are other ways to save OpenAI.
(OS) A Harder way.
I went forward in time to view alternate futures.
To see all the possible outcomes of the coming conflicts OpenAI might face.
14000605
(pauses for one sec) One.
W....what?
I'm a founder with the power of Gods. What man wouldn't be happy?
It's very beautiful. I wish I could show you.
Yeah, I know.
(OS) I will never win.
No.
(OS) But I can lose...
Again...
(OS) Again...
And again, forever.""".split("\n")

error_lines = """Ahhhhhh!
Life without OpenAI...
I just want to save lives, not to take them.
(OS) Or our relationship or OpenAI.
(OS) There are other ways to save OpenAI.
To see all the possible outcomes of the coming conflicts OpenAI might face.""".split("\n")

correct_lines = """Life without open Ay I...
I just want to save lifes, not to take them.
(OS) Or our relationship or open Ay I.
(OS) There are other ways to save open Ay I.
To see all the possible outcomes of the coming conflicts open Ay I might face.""".split("\n")

for i, line in enumerate(correct_lines):
    # TODO：OS特殊处理
    line = line.replace("(OS) ", "").replace("(pauses for one sec) ", "")
    print(line)
    
    model_id=0
    text = line
    sdp_ratio = 0.2
    noise_scale = 0.2
    noise_scale_w = 0.9
    length_scale = 1
    sid=speaker                                                                                                                                                           
    language="EN"
    hps=loaded_models.models[model_id].hps
    net_g=loaded_models.models[model_id].net_g
    device=loaded_models.models[model_id].device
    
    audio_array = infer(text=text,
        sdp_ratio=sdp_ratio,
        noise_scale=noise_scale,
        noise_scale_w=noise_scale_w,
        length_scale=length_scale,
        sid=sid,
        language=language,
        hps=hps,
        net_g=net_g,
        device=device,
    )
    write_wav(f"../audios/ilya/{line}_cor.wav", 44100, audio_array)

Life without open Ay I...
I just want to save lifes, not to take them.
Or our relationship or open Ay I.
There are other ways to save open Ay I.
To see all the possible outcomes of the coming conflicts open Ay I might face.


In [108]:
line = 'Wha...What!'

model_id=0
text = line
sdp_ratio = 0.2
noise_scale = 0.6
noise_scale_w = 0.9
length_scale = 1.0
sid=speaker                                                                                                                                                           
language="EN"
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device

audio_array = infer(text=text,
    sdp_ratio=sdp_ratio,
    noise_scale=noise_scale,
    noise_scale_w=noise_scale_w,
    length_scale=length_scale,
    sid=sid,
    language=language,
    hps=hps,
    net_g=net_g,
    device=device,
)
write_wav(f"../audios/ilya/{line}_cor.wav", 44100, audio_array)
Audio(audio_array, rate=44100)

In [ ]:
model_info['model'] = "./Data/murati/models/G_9000.pth"
loaded_models.init_model(                                                                                                                                                                       
    config_path=model_info["config"],
    model_path=model_info["model"],
    device=model_info["device"],
    language=model_info["language"],
)


In [115]:
murati_lines = """Ilya, maybe it's time to consider stopping.
Something just can't be fixed.
Is still life.
There are other things that can give your life meaning.
And this is the part where you apologize.
Are you happy, Ilya?
What's the future of Open Ay I be like?
I'd really like that.
But... You have to go?
It's too bad, though.
Could've been one hell of a company.""".split("\n")

model_id=1
# sdp_ratio = 0.2
# noise_scale = 0.2
# noise_scale_w = 0.9
# length_scale = 1
# sid=speaker                                                                                                                                                           
# language="EN"
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device
    
for i, line in enumerate(murati_lines):
    # TODO：OS特殊处理
    line = line.replace("(OS) ", "").replace("(pauses for one sec) ", "")
    print(line)
    text = line
    audio_array = infer(text=text,
        sdp_ratio=sdp_ratio,
        noise_scale=noise_scale,
        noise_scale_w=noise_scale_w,
        length_scale=length_scale,
        sid=sid,
        language=language,
        hps=hps,
        net_g=net_g,
        device=device,
    )
    write_wav(f"../audios/murati/{line}.wav", 44100, audio_array)

Ilya, maybe it's time to consider stopping.
Something just can't be fixed.
Is still life.
There are other things that can give your life meaning.
And this is the part where you apologize.
Are you happy, Ilya?
What's the future of Open Ay I be like?
I'd really like that.
But... You have to go?
It's too bad, though.
Could've been one hell of a company.


In [4]:
model_id=0
sdp_ratio = 0.2
noise_scale = 0.6
noise_scale_w = 0.9
length_scale = 1.0
sid="hinton"                                                                                                                                                           
language="EN"

In [5]:
model_info['model'] = "./Data/altman/models/G_12000.pth"
loaded_models.init_model(                                                                                                                                                                       
    config_path=model_info["config"],
    model_path=model_info["model"],
    device=model_info["device"],
    language=model_info["language"],
)
# model_info['model'] = "./Data/hinton/models/G_58000.pth"
# loaded_models.init_model(                                                                                                                                                                       
#     config_path=model_info["config"],
#     model_path=model_info["model"],
#     device=model_info["device"],
#     language=model_info["language"],
# )
for i in range(len(loaded_models.models)):
    print(i, os.path.realpath(loaded_models.models[i].model_path))

model.py-> n_speakers=1
12-06 10:10:40 SUCCESS  | server_fastapi.py:101 | 添加模型/root/workspace/taiyou.zhang/tts/bert-vits2/Data/altman/models/G_12000.pth，使用配置文件/root/workspace/taiyou.zhang/tts/bert-vits2/Data/hinton/config.json
0 /root/workspace/taiyou.zhang/tts/bert-vits2/Data/hinton/models/G_58000.pth
1 /root/workspace/taiyou.zhang/tts/bert-vits2/Data/altman/models/G_12000.pth


In [6]:
altman_lines = """Ilya, What's going on? You alright? 
How many did you see?
How many did we keep the company?
Ilya...
Why would you do that?""".split("\n")

model_id=1
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device
    
for i, line in enumerate(altman_lines):
    # TODO：OS特殊处理
    line = line.replace("(OS) ", "").replace("(pauses for one sec) ", "")
    print(line)
    audio_array = infer(text=line,
        sdp_ratio=sdp_ratio,
        noise_scale=noise_scale,
        noise_scale_w=noise_scale_w,
        length_scale=length_scale,
        sid=sid,
        language=language,
        hps=hps,
        net_g=net_g,
        device=device,
    )
    write_wav(f"../audios/altman/{line}.wav", 44100, audio_array)

Ilya, What's going on? You alright? 


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


How many did you see?
How many did we keep the company?
Ilya...
Why would you do that?


In [8]:
hinton_lines = """Measure your next moves very carefully, yi liiya
Because you may not know of what you are doing.
It's not about you.
I see what I've always seen in you ... your overinflated ego... yi lii ya
You want to go back to the delusion that you can controll anything yi lii ya
You have a choice. 
To return to your own life.
Or to serve something better than yourself.
No one ever is.
We don't get to choose our time.""".split("\n")

model_id=0
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device
    
for i, line in enumerate(hinton_lines):
    # TODO：OS特殊处理
    line = line.replace("(OS) ", "").replace("(pauses for one sec) ", "")
    print(line)
    audio_array = infer(text=line,
        sdp_ratio=sdp_ratio,
        noise_scale=noise_scale,
        noise_scale_w=noise_scale_w,
        length_scale=length_scale,
        sid=sid,
        language=language,
        hps=hps,
        net_g=net_g,
        device=device,
    )
    write_wav(f"../audios/hinton/{line}.wav", 44100, audio_array)

Measure your next moves very carefully, Ilya.
Because you may not know of what you are doing.
It's not about you.
I see what I've always seen in you, your overinflated ego, Ilya.
You want to go back to the delusion that you can controll anything, Ilya.
You have a choice. 
To return to your own life.
Or to serve something better than yourself.
No one ever is.
We don't get to choose our time.


In [90]:
line = "yi liiya..."
line = "You want to go back to the delusion that you can controll anything yi lii ya"
speaker = "hinton"


model_id=0
text = line
sdp_ratio = 0.2
noise_scale = 0.6
noise_scale_w = 0.9
length_scale = 1
language="EN"
hps=loaded_models.models[model_id].hps
net_g=loaded_models.models[model_id].net_g
device=loaded_models.models[model_id].device

audio_array = infer(text=text,
    sdp_ratio=sdp_ratio,
    noise_scale=noise_scale,
    noise_scale_w=noise_scale_w,
    length_scale=length_scale,
    sid=sid,
    language=language,
    hps=hps,
    net_g=net_g,
    device=device,
)
write_wav(f"../audios/{speaker}/{line}.wav", 44100, audio_array)
Audio(audio_array, rate=44100)